medallion_staging--> bronze table


In [0]:
spark.read.csv(path="/Volumes/dev_catalog/default/retail_staging/customers/customers.csv",
               header=True,
               inferSchema=True).write.format("delta").mode("overwrite").saveAsTable("dev_catalog.bronze.customers")

In [0]:
spark.read.table("dev_catalog.bronze.customers").display()

In [0]:
spark.read.csv(path="/Volumes/dev_catalog/default/retail_staging/orders/",
               header=True,
               inferSchema=True).write.format("delta").mode("overwrite").saveAsTable("dev_catalog.bronze.orders")

In [0]:
spark.read.table("dev_catalog.bronze.orders").display()